In [5]:
year = 2024; day = 12

In [14]:
from aocd import get_data, submit

data = get_data(year=year, day=day)

data = data.strip()
data = [d for d in data.split()]
W = len(data[0])
H = len(data)
assert W == H

In [15]:
directions_map = {
    "W": (0, -1),
    "NW": (-1, -1),
    "N": (-1, 0),
    "NE": (-1, 1),
    "E": (0, 1),
    "SE": (1, 1),
    "S": (1, 0),
    "SW": (1, -1)
}

def in_bounds(y, x):
    return 0 <= y < H and 0 <= x < W

def get_rel_coords(y, x, dir):
    dy, dx = directions_map[dir]
    return y+dy, x+dx

def loc_data(y, x):
    return data[y][x] if in_bounds(y, x) else ""

def loc_data_rel(y, x, dir):
    return loc_data(*get_rel_coords(y, x, dir))

In [16]:
from collections import defaultdict
directions = ["W", "N", "E", "S"]

area = defaultdict(int)
edges = defaultdict(int)

def flood_fill(y, x, C):
    edges = 0
    area = set()
    positions = [(y, x)]
    visited = set()

    for _ in range(1000):
        new_positions = set()
        for p in positions:
            y, x = p
            if p in visited:
                continue
            visited.add(p)
            if loc_data(y, x) == C:
                area.add(p)

            for dir in directions:
                new_p = get_rel_coords(y, x, dir)
                val = loc_data(*new_p)
                if val == C:
                    new_positions.add(new_p)
                else:
                    edges += 1

        if not new_positions:
            return edges, area

        positions = list(new_positions.copy())
    raise ValueError("Too many iterations")

In [17]:
visited = set()
all_fields = []
for y in range(H):
    for x in range(W):
        c = data[y][x]
        if (y, x) in visited:
            continue
        edges, members = flood_fill(y, x, c)
        visited.update(members)

        all_fields.append((c, edges, members))

In [18]:
corners_dirs = [
    ["W", "NW", "N"],
    ["N", "NE", "E"],
    ["E", "SE", "S"],
    ["S", "SW", "W"]
]

def count_corners(members):
    corners = 0
    members = sorted(list(members))
    for y, x in members:
        C = data[y][x]

        for corner_dir in corners_dirs:
            corner_neigh = [
                loc_data_rel(y, x, dir) == C
                for dir in corner_dir
            ]
            # . C
            # C C
            # interior corner, W NW N example
            if corner_neigh == [True, False, True]:
                corners += 1
            # . .
            # . C
            # exterior corner, W NW N example
            if corner_neigh == [False, False, False]:
                corners += 1
            # C .
            # . C
            # exterior corner touching another field, W NW N example
            if corner_neigh == [False, True, False]:
                corners += 1

    return corners


In [19]:
ans1 = 0
for c, e, m in all_fields:
    ans1 += e * len(m)
submit(ans1, part="a", year=year, day=day)

Part a already solved with same answer: 1344578


In [20]:
ans2 = 0
for C, edges, members in all_fields:
    ans2 += len(members) * count_corners(members)
submit(ans2, part="b", year=year, day=day)

Part b already solved with same answer: 814302
